In [1]:
# variables with suffix '_n' denote that nudged data are being used
# variables with prefix 'd' denote derivative

# dependencies
using Base.Iterators
using MovingWeightedLeastSquares
using Plots
using Cubature
using Interpolations

# initialization of PyPlot plotting backend
pyplot();

In [197]:
# initialize the dataset in this Jupyter notebook cell

# from, to, step
from = -1.95;
to = 1.95;
step = 0.975/2^3;
# a random number in interval (-ng/2, ng/2) will be added to each x in xs
ng = .2;
# function to be approximated
f = (x, y) -> exp(-(x^2 + y^2));
df = (x, y) -> 4 * x * y * exp(-(x^2 + y^2));
# weight function
w = (d, e) -> exp(-d^2/0.28^2);
# degree of used polynomial for approximation
deg = 4;
# eps of the weight function
eps = 2.5*step;
# attempts for time measurement
time_tries = 10;

In [198]:
rg = from:step:to;
tozipxs = collect(from:step:to);
xs = transpose(hcat(collect.(collect(product(tozipxs,tozipxs)))[:]...));

# comment the line below, if repeated evaluation with the same nudges is needed
rnd_n = ng * rand(size(xs)) - ng/2;
xs_n = xs .+ rnd_n;

fs = [f(xs[i, :]...) for i in 1:size(xs, 1)];
fs_n = [f(xs_n[i, :]...) for i in 1:size(xs_n, 1)];
dfs = [df(xs[i, :]...) for i in 1:size(xs_n, 1)];
dfs_n = [df(xs_n[i, :]...) for i in 1:size(xs_n, 1)];
size(xs, 1)

1089

In [199]:
# creation of approximation objects
nv = mwlsNaive(xs, fs, eps, w, maxDegree = deg);
# nv_n = mwlsNaive(xs_n, fs_n, eps, w, maxDegree = deg);
kd = mwlsKd(xs, fs, eps, w, maxDegree = deg);
# kd_n = mwlsKd(xs_n, fs_n, eps, w, maxDegree = deg);
cll = mwlsCll(xs, fs, eps, w, maxDegree = deg);
# cll_n = mwlsCll(xs_n, fs_n, eps, w, maxDegree = deg);

In [200]:
function mass_approx(obj::MwlsObject, xs::Array{T}) where {T <: Real}
    [obj(xs[i, :]) for i in 1:size(xs, 1)]
end

mass_approx (generic function with 1 method)

In [201]:
function mass_diff(obj::MwlsObject, xs::Array{T}) where {T <: Real}
    [mwlsDiff(obj, xs[i, :], (1, 1)) for i in 1:size(xs, 1)]
end

mass_diff (generic function with 1 method)

In [202]:
# construction of approximation at each input vector
nvs = mass_approx(nv, xs);

In [203]:
@time for i in 1:time_tries
    mass_approx(nv, xs)
end

419.341827 seconds (1.76 G allocations: 44.012 GiB, 6.04% gc time)


In [204]:
kds = mass_approx(kd, xs);

In [205]:
@time for i in 1:time_tries
    mass_approx(kd, xs)
end

292.071572 seconds (1.36 G allocations: 30.841 GiB, 6.13% gc time)


In [206]:
clls = mass_approx(cll, xs);

In [207]:
@time for i in 1:time_tries
    mass_approx(cll, xs)
end

295.283326 seconds (1.38 G allocations: 32.628 GiB, 6.47% gc time)


In [208]:
dnvs = mass_diff(nv, xs);

In [209]:
@time for i in 1:time_tries
    mass_diff(nv, xs)
end

434.343218 seconds (1.76 G allocations: 44.053 GiB, 6.02% gc time)


In [210]:
dkds = mass_diff(kd, xs);

In [211]:
@time for i in 1:time_tries
    mass_diff(kd, xs);
end

294.011937 seconds (1.36 G allocations: 30.881 GiB, 6.30% gc time)


In [212]:
dclls = mass_diff(cll, xs);

In [213]:
@time for i in 1:time_tries
    mass_diff(cll, xs);
end

297.926290 seconds (1.38 G allocations: 32.669 GiB, 6.56% gc time)


In [214]:
function interpolations_mass_approx(obj, xs::Array{T}) where {T <: Real}
    [obj[xs[i, :]...] for i in 1:size(xs, 1)]
end

interpolations_mass_approx (generic function with 1 method)

In [215]:
# init of the interpolation object from Interpolations.jl
itp = scale(interpolate([f(x1, x2) for x1 in rg, x2 in rg], BSpline(Cubic(Line())), OnGrid()), rg, rg);

In [216]:
# result of approximation
itps = interpolations_mass_approx(itp, xs);

In [217]:
@time for i in 1:time_tries
    interpolations_mass_approx(itp, xs)
end

  0.004251 seconds (71.17 k allocations: 2.002 MiB)
